# Novelty detection

Let's try to detect if the in situ data point are considered as outliers regarding the laboratory cultures. If so, it justify the need to create new category to predict from the in situ data, instead of the dominant taxa.

In [ ]:
# import lab and insitu dataset

insitu = pd.read_csv("/home/flavien/Documents/these/mf/Boussole/Data/labo/results_reshape.csv")

#filter data with less than 0.2 chla
insitu_conc = insitu[insitu['chla_conc_spectro'] > 0.2]

#select the cols we want
insitu_sel = insitu_conc[["440nm_mean", "470nm_mean", "532nm_mean", "strain", "replicate"]]

#remove the NaN
insitu_sel = insitu_sel.dropna()

#rename columns to make it easier to work with
insitu_sel = insitu_sel.rename(columns = {"440nm_mean": "f440",
                                    "470nm_mean": "f470",
                                    "532nm_mean": "f532"})

#compute the ratio between each excitation channel
insitu_sel = insitu_sel.assign(f440_f470 = lambda x: x['f440']/x['f470'],
                f532_f470 = lambda x: x['f532']/x['f470'],
                f532_f440 = lambda x: x['f532']/x['f440'])